### Please install the required Python modules/SDKs

In [1]:
! activate ai-azure-c1

import sys

sys.path.append("/opt/conda/envs/ai-azure-c1/lib/python3.8/site-packages")

Could not find conda environment: ai-azure-c1
You can list all discoverable environments with `conda info --envs`.



## Importing Python modules for the demo

In [2]:
import requests
import os, time, uuid

In [3]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials

## Make sure you have the correct Training Endpoint, Key and Resource ID

In [4]:
TRAINING_ENDPOINT = "https://nayanacustomvision.cognitiveservices.azure.com/"
training_key = "ce08598caf404621b37136119dccd921"
training_resource_id = "/subscriptions/bb272072-9c6d-4e28-b814-947814c3e6ef/resourceGroups/nayana-ai-msft-azure/providers/Microsoft.CognitiveServices/accounts/nayanacustomvision"

In [5]:
training_credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(TRAINING_ENDPOINT, training_credentials)

### Getting a list of all Custom Vision projects 

In [6]:
all_project = trainer.get_projects()

In [7]:
for each_project in all_project:
    print("Project ID: ",each_project.id)
    print("Project Name: ",each_project.name)

Project ID:  c4c324e8-4f9e-49e3-b4be-797b38a686de
Project Name:  cfd70f0c-a407-4410-b6ab-bc0632cd547e
Project ID:  ddd61aae-3f1f-4196-94d6-7b4badc76ea7
Project Name:  Automated-Passenger-Boarding-Kiosk


### Select the project ID that you are interested in exporting. Make sure the project type is set to "Compact".

In [8]:
project_id = all_project[0].id
print("Project ID: ", project_id)

Project ID:  c4c324e8-4f9e-49e3-b4be-797b38a686de


In [9]:
all_iterations = trainer.get_iterations(project_id)

In [10]:
for index, each_iteration in enumerate(all_iterations):
    print("|--------- Iteration [{}] --------".format(index))
    print("Iteration ID: ",each_iteration.id)
    print("Iteration Name: ",each_iteration.name)
    print("Iteration Exportable: ",each_iteration.exportable)
    print("-----------------------------------------------------")

|--------- Iteration [0] --------
Iteration ID:  20d0fcb5-25eb-4808-bcf3-5a9cab295eec
Iteration Name:  Iteration 2
Iteration Exportable:  True
-----------------------------------------------------
|--------- Iteration [1] --------
Iteration ID:  31106c08-c783-42e1-a43a-fb99e3b74928
Iteration Name:  Iteration 1
Iteration Exportable:  False
-----------------------------------------------------


### Above please verify that your project-> Iteration must be exportalble
- Iteration Exportable: True

Make sure there are exportable iterations. If none of the iteration is exportable, follow these steps:
* Go to the Custom Vision portal
* Select a project and change your project setting to support the compact domain
* Create another iteration for model training

You can check the *"Azure Custom Vision Portal Demo 5: Export a Model"* video for how to perform these steps.

Then you can rerun the cells above.

In [11]:
for index, each_iteration in enumerate(all_iterations):
    model_perf = trainer.get_iteration_performance(project_id, each_iteration.id)
    print("----- Model Perormance {} -------- ".format(index))
    print("Iteration ID: ",each_iteration.id)
    print("Precision: {}   Recall: {} -------- ".format(model_perf.precision, model_perf.recall))
    print("-----------------------------------------------------")

----- Model Perormance 0 -------- 
Iteration ID:  20d0fcb5-25eb-4808-bcf3-5a9cab295eec
Precision: 1.0   Recall: 0.23809524 -------- 
-----------------------------------------------------
----- Model Perormance 1 -------- 
Iteration ID:  31106c08-c783-42e1-a43a-fb99e3b74928
Precision: 1.0   Recall: 0.074074075 -------- 
-----------------------------------------------------


### Select the project ID that you are interested in exporting. Make sure it is exportable.

In [12]:
selected_iteration_id = all_iterations[0].id
print(selected_iteration_id)

20d0fcb5-25eb-4808-bcf3-5a9cab295eec


### Platform Choices are
- CoreML, TensorFlow, DockerFile, ONNX, VAIDK, and OpenVino

### Flavor Choices are
- Linux, Windows, ONNX10, ONNX12, ARM, TensorFlowNormal, and TensorFlowLite.

### raw = True|False
- If True, you can get raw JSON response along with the object model response

# Start the model export process

In [13]:
platform = "TensorFlow"
flavor = "TensorFlowLite"

In [14]:
export_process = trainer.export_iteration(project_id, selected_iteration_id, platform, flavor, raw=True)

CustomVisionErrorException: 20d0fcb5-25eb-4808-bcf3-5a9cab295eec is already queued for export

In [ ]:
print(export_process.output)

In [ ]:
print(export_process.output.status)

In [ ]:
### Code snippet is from Azure SDK and Documentation
### https://docs.microsoft.com/en-us/azure/cognitive-services/custom-vision-service/export-programmatically
### This step may take a long time 
while (export_process.output.status == "Exporting"):
    print ("Waiting 10 seconds...")
    time.sleep(10)
    exports = trainer.get_exports(project_id, selected_iteration_id)
    for e in exports:
        if e.platform == export_process.output.platform and e.flavor == export_process.output.flavor:
            export = e
            break
    print("Export status is: ", export_process.output.status)

In [ ]:
print(export_process.output.status)

In [ ]:
print(export_process.output.download_uri)

In [ ]:
## Downloading the model from url
if export_process.output.status == "Done":
    # Ready to Download
    model_export_file = requests.get(export_process.output.download_uri)
    with open("birds_flowers_model_tensorflow.zip", "wb") as file:
        file.write(model_export_file.content)

# Collecting Export Status by project ID and iteration ID

In [ ]:
exports_offline = trainer.get_exports(project_id, selected_iteration_id)

In [ ]:
for e in exports_offline:
    print(e)

In [ ]:
for e in exports_offline:
    if e.status == 'Done':
        print("Downloading model from URL: ", e.download_uri)
        model_export_file = requests.get(e.download_uri)
        with open("birds_flowers_model_tensorflow.zip", "wb") as file:
            file.write(model_export_file.content)

In [ ]:
!ls -l birds_flowers_model_tensorflow.zip